# Monitoring & Actuating GPIOS 
By Marcelo Rovai @24Oct25

## Test Setup

In [1]:
import time
import board
import adafruit_dht

## Playing with Sensors/Actuators

### Import libraries, instantiate and initialize sensors/actuators

In [2]:
import time
import datetime

# Adafruit DHT library (Temperature/Humidity)
import board
import adafruit_dht
DHT22Sensor = adafruit_dht.DHT22(board.D16)

# BMP library (Pressure/Temperature)
import adafruit_bmp280
i2c = board.I2C()
bmp280Sensor = adafruit_bmp280.Adafruit_BMP280_I2C(i2c, address = 0x76)
bmp280Sensor.sea_level_pressure = 1013.25

# LEDs
from gpiozero import LED

ledRed = LED(13)
ledYlw = LED(19)
ledGrn = LED(26)

ledRed.off()
ledYlw.off()
ledGrn.off()

# Push-Button
from gpiozero import Button
button = Button(20)

### Create a function to get GPIO status

In [3]:
def led_status():
    ledRedSts = ledRed.is_lit
    ledYlwSts = ledYlw.is_lit
    ledGrnSts = ledGrn.is_lit 
    return ledRedSts, ledYlwSts, ledGrnSts

In [4]:
led_status()

(False, False, False)

In [5]:
# Get GPIO status data 
def getGpioStatus():
    global timeString
    global buttonSts
    global ledRedSts
    global ledYlwSts
    global ledGrnSts

    # Get time of reading
    now = datetime.datetime.now()
    timeString = now.strftime("%Y-%m-%d %H:%M")
    
    # Read GPIO Status
    buttonSts = button.is_pressed
    ledRedSts = ledRed.is_lit
    ledYlwSts = ledYlw.is_lit
    ledGrnSts = ledGrn.is_lit 

### Function to Print the Status

In [6]:
# Print GPIO status data 
def PrintGpioStatus():
    print ("Local Station Time: ", timeString)
    print ("Led Red Status:     ", ledRedSts)
    print ("Led Yellow Status:  ", ledYlwSts)
    print ("Led Green Status:   ", ledGrnSts)
    print ("Push-Button Status: ", buttonSts)

#### Turning-On the LEDs

In [7]:
ledRed.on()
ledYlw.on()
ledGrn.on()

In [8]:
getGpioStatus()
PrintGpioStatus()

Local Station Time:  2025-10-28 10:50
Led Red Status:      True
Led Yellow Status:   True
Led Green Status:    True
Push-Button Status:  False


#### Pressing the button

In [9]:
getGpioStatus()
PrintGpioStatus()

Local Station Time:  2025-10-28 10:50
Led Red Status:      True
Led Yellow Status:   True
Led Green Status:    True
Push-Button Status:  True


#### Turning off the LEDs

In [10]:
ledRed.off()
ledYlw.off()
ledGrn.off()

getGpioStatus()
PrintGpioStatus()

Local Station Time:  2025-10-28 10:50
Led Red Status:      False
Led Yellow Status:   False
Led Green Status:    False
Push-Button Status:  False


### Function to Turn-On Individual LEDs

In [11]:
# Acting on GPIOs and printing Status
def controlLeds(r, y, g):
    if (r):
        ledRed.on()
    else:
        ledRed.off()        
    if (y):
        ledYlw.on()
    else:
        ledYlw.off() 
    if (g):
        ledGrn.on()
    else:
        ledGrn.off() 
    
    getGpioStatus()
    PrintGpioStatus()

In [12]:
controlLeds(1, 1, 1)

Local Station Time:  2025-10-28 10:50
Led Red Status:      True
Led Yellow Status:   True
Led Green Status:    True
Push-Button Status:  False


In [13]:
controlLeds(0, 1, 0)

Local Station Time:  2025-10-28 10:50
Led Red Status:      False
Led Yellow Status:   True
Led Green Status:    False
Push-Button Status:  False


In [14]:
controlLeds(0, 0, 0)

Local Station Time:  2025-10-28 10:50
Led Red Status:      False
Led Yellow Status:   False
Led Green Status:    False
Push-Button Status:  False


#### Alternative Function to control LEDs

In [15]:
def control_leds(red, yellow, green):
    ledRed.on() if red else ledRed.off()
    ledYlw.on() if yellow else ledYlw.off()
    ledGrn.on() if green else ledGrn.off()

In [17]:
control_leds(1,1,1)

In [18]:
control_leds(0,0,0)

### Getting and displaying Sensor Data

#### Read data from BMP280

In [19]:

def bmp280GetData(real_altitude):
    
    temp = bmp280Sensor.temperature
    pres = bmp280Sensor.pressure
    alt =  bmp280Sensor.altitude
    presSeaLevel = pres / pow(1.0 - real_altitude/44330.0, 5.255) 
    
    temp = round (temp, 1)
    pres = round (pres, 2) # absolute pressure in mbar
    alt = round (alt)
    presSeaLevel = round (presSeaLevel, 2) # absolute pressure in mbar
    
    return temp, pres, alt, presSeaLevel

In [20]:
temp, pres, alt, presSeaLevel = bmp280GetData(960)
temp, pres, alt, presSeaLevel

(22.5, 910.25, 895, 1021.24)

In [25]:
def DHT22GetData():
    
    tempDHT =  DHT22Sensor.temperature
    humDHT =  DHT22Sensor.humidity

    if humDHT is not None and tempDHT is not None:
        tempExt = round (tempDHT)
        humExt = round (humDHT)
    
    return tempDHT, humDHT

In [26]:
tempDHT, humDHT = DHT22GetData()
tempDHT, humDHT

(21.7, 36.4)

#### Get data (from local sensors)

In [23]:
def getSensorData(altReal=0):
    global timeString
    global humExt
    global tempLab
    global tempExt
    global presSL
    global altLab
    global presAbs
    global buttonSts
	
    # Get time of reading
    now = datetime.datetime.now()
    timeString = now.strftime("%Y-%m-%d %H:%M")
    
    tempLab, presAbs, altLab, presSL = bmp280GetData(altReal) 
    
    tempDHT =  DHT22Sensor.temperature
    humDHT =  DHT22Sensor.humidity
	
    if humDHT is not None and tempDHT is not None:
        tempExt = round (tempDHT)
        humExt = round (humDHT)

#### Display important data on-screen

In [27]:
def printData():
    print ("Local Station Time:             ", timeString)
    print ("External Air Temperature (DHT): ", tempExt, "oC")
    print ("External Air Humidity    (DHT): ", humExt, "%")
    print ("Station Air Temperature  (BMP): ", tempLab, "oC")
    print ("Sea Level Air Pressure:         ", presSL, "mBar")
    print ("Absolute Station Air Pressure:  ", presAbs, "mBar")
    print ("Station Measured Altitude:      ", altLab, "m")

In [28]:
real_altitude = 960 # real altitude of where the BMP280 is installed
getSensorData(real_altitude)
printData()

Local Station Time:              2025-10-28 10:54
External Air Temperature (DHT):  22 oC
External Air Humidity    (DHT):  36 %
Station Air Temperature  (BMP):  22.5 oC
Sea Level Air Pressure:          1021.26 mBar
Absolute Station Air Pressure:   910.27 mBar
Station Measured Altitude:       895 m


### Function to simply collect data

In [29]:
def collect_data():
    try:
        temperature_dht = DHT22Sensor.temperature
        humidity = DHT22Sensor.humidity
        temperature_bmp = bmp280Sensor.temperature
        pressure = bmp280Sensor.pressure
        button_pressed = button.is_pressed
        return temperature_dht, humidity, temperature_bmp, pressure, button_pressed
    except RuntimeError:
        return None, None, None, None, None

In [30]:
collect_data()

(21.6, 36.7, 22.5376953125, 910.2393920160168, False)